# 01/20

%matplotlib inline

In [1]:
description = 'train_mobilenet_from_pretrain_v3_add_valence_arousal_batch32'

In [2]:

import os
import numpy as np
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import cv2
from PIL import Image
from random import shuffle
import math

import pandas as pd

import pickle

from sklearn.svm import SVC,LinearSVC
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier,ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import preprocessing

#from scipy.misc import imread, imresize



In [3]:
import tqdm
import json

In [4]:
train_df = pd.read_csv('../../data/Manually_Annotated_file_lists/training_face_mesh_crop.csv')
train_df.subDirectory_filePath = '../../data/Manually_Annotated_Images_FaceMesh_Cropped/' + train_df.subDirectory_filePath

#train_df_2 = pd.read_csv('../../data/Automatically_annotated_file_list/automatically_annotated_face_mesh_crop.csv')
#train_df_2.subDirectory_filePath = '../../data/Automatically_Annotated_Images_FaceMesh_Cropped/' + train_df_2.subDirectory_filePath
#train_df = train_df.append(train_df_2)
#del train_df_2

train_df = train_df[train_df['have_facemesh']]

In [5]:
from eason_utils import DataFrameBatchIterator
from eason_utils import lprint, now_time_string, log_file_name, change_log_file_name

change_log_file_name(f'''{log_file_name.replace('.log', '')}_{description}.log''')

log_file_name='01_22_23:42:03.log'


In [6]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
config = tf.compat.v1.ConfigProto(gpu_options = tf.compat.v1.GPUOptions(allow_growth = True))
sess = tf.compat.v1.Session(config = config)

In [7]:
mobilenet_pretrained = tf.keras.models.load_model("../models/affectnet_emotions/mobilenet_7.h5")

In [8]:
mobilenet_pretrained.trainable = False
mobilenet_output = mobilenet_pretrained.get_layer("global_pooling").output

In [9]:
hidden_layers = [256]

In [10]:
valaence_feat = mobilenet_output
for size in hidden_layers:
    valence_feat = tf.keras.layers.Dense(size, activation = 'relu', name = f'feat_valence')(valaence_feat)
outputs_valence = (tf.keras.layers.Dense(1, activation = 'sigmoid', name = 'outputs_valence')(valence_feat) * 4) - 2

arousal_feat = mobilenet_output
for size in hidden_layers:
    arousal_feat = tf.keras.layers.Dense(size, activation = 'relu', name = f'feat_arousal')(arousal_feat)
outputs_arousal = (tf.keras.layers.Dense(1, activation = 'sigmoid', name = 'outputs_arousal')(arousal_feat) * 4) - 2


In [11]:
model = tf.keras.Model(inputs=mobilenet_pretrained.input,
                       outputs=(outputs_valence, outputs_arousal, mobilenet_pretrained.output) , name="mobilenet_train")

In [12]:
model.summary()

Model: "mobilenet_train"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 225, 225, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1 (Conv2D)                 (None, 112, 112, 32  864         ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                    

In [13]:
# Instantiate an optimizer.
optimizer = tf.keras.optimizers.Adam()
# Instantiate a loss function.
MSE_loss = tf.keras.losses.MeanSquaredError()
SCC_loss = tf.keras.losses.SparseCategoricalCrossentropy()

In [14]:
batch_size = 32
epochs = 4

logs = []

In [ ]:
from telegram_notifier import send_message
import time
start_time = time.time()


for epoch in range(epochs):
    total_loss = 0
    total_step = 0
    total_emotion_correct = 0
    total_valence_loss = 0
    total_arousal_loss = 0
    total_emotion_loss = 0
    total_process = 0
    for step, row in enumerate(DataFrameBatchIterator(train_df, batch_size=batch_size)):

        imgs = row.subDirectory_filePath.apply(lambda x: cv2.resize(
            cv2.cvtColor(cv2.imread(x), cv2.COLOR_BGR2RGB), (224, 224)))
        img_array = np.array(list(imgs))

        y_valence = np.array(row.valence)
        y_arousal = np.array(row.arousal)
        y_emotion = np.array(row.expression)

        with tf.GradientTape() as tape:
            logits = model(img_array, training=True)

            pred_valence = logits[0]
            pred_arousal = logits[1]
            pred_emotion = logits[2]

            valence_loss = MSE_loss(y_valence, pred_valence)
            arousal_loss = MSE_loss(y_arousal, pred_arousal)
            emotion_loss = SCC_loss(y_emotion, pred_emotion)

            loss = valence_loss + arousal_loss # + emotion_loss

            grads = tape.gradient(loss, model.trainable_weights)

            optimizer.apply_gradients(zip(grads, model.trainable_weights))
        total_loss += float(loss)
        total_step += 1
        
        total_process += len(row)

        valence_loss = float(valence_loss)
        arousal_loss = float(arousal_loss)
        emotion_loss = float(emotion_loss)
        
        total_valence_loss += valence_loss
        total_arousal_loss += arousal_loss
        total_emotion_loss += emotion_loss
        
        emotion_correct = int(sum(pred_emotion.numpy().argmax(axis = 1) == y_emotion))
        total_emotion_correct += emotion_correct
        
        log = {
            'emotion_correct': emotion_correct,
            'valence_loss': valence_loss,
            'arousal_loss': arousal_loss,
            'emotion_loss': emotion_loss,
            'time_take': time.time() - start_time,
            'pred_valence': list(map(lambda x: float(x), np.around(np.array(pred_valence).reshape(-1), 3))),
            'pred_arousal': list(map(lambda x: float(x), np.around(np.array(pred_arousal).reshape(-1), 3))),
        }
        log = json.dumps(log)
        lprint(log)
    save_model_path = f"models/{now_time_string}_{description}_epoch{epoch}_batch_{batch_size}"
    model.save(save_model_path)
    lprint(f"Save {save_model_path}")
    send_message(f"Save {save_model_path}\nepoch {epoch} is finish")
    log = {
        'model_path': save_model_path,
        'total_emotion_correct': total_emotion_correct,
        'total_loss': total_loss,
        'total_valence_loss': total_valence_loss,
        'total_arousal_loss': total_arousal_loss,
        'total_emotion_loss': total_emotion_loss,
        'total_process': total_process,
        'time_take': time.time() - start_time,
    }
    log = json.dumps(log)
    lprint(log)

{"emotion_correct": 2, "valence_loss": 1.648931622505188, "arousal_loss": 2.094984292984009, "emotion_loss": NaN, "time_take": 2.59694504737854, "pred_valence": [0.18400000035762787, 0.003000000026077032, -0.1120000034570694, 0.05400000140070915, -0.15399999916553497, -0.1850000023841858, 0.028999999165534973, -0.4180000126361847, 0.15299999713897705, -0.35199999809265137, 0.032999999821186066, -0.11299999803304672, -0.06300000101327896, -0.06400000303983688, -0.08799999952316284, -0.33500000834465027, -0.21899999678134918, -0.0949999988079071, -0.7870000004768372, -0.2460000067949295, -0.11500000208616257, 0.125, -0.08100000023841858, 0.15600000321865082, -0.10599999874830246, -0.34599998593330383, -0.24799999594688416, 0.26899999380111694, -0.09000000357627869, 0.10700000077486038, -0.16300000250339508, -0.17800000309944153], "pred_arousal": [0.3409999907016754, 0.210999995470047, 0.052000001072883606, 0.21799999475479126, 0.2370000034570694, 0.04500000178813934, -0.18000000715255737

{"emotion_correct": 1, "valence_loss": 1.6851770877838135, "arousal_loss": 1.3010191917419434, "emotion_loss": NaN, "time_take": 3.793994665145874, "pred_valence": [0.1589999943971634, 0.9760000109672546, 0.6959999799728394, 0.014999999664723873, 0.9269999861717224, 0.34200000762939453, 0.20600000023841858, 0.2770000100135803, 0.7120000123977661, -0.0820000022649765, -0.0560000017285347, -0.00800000037997961, 0.2460000067949295, 0.3009999990463257, -0.18199999630451202, 0.05000000074505806, 0.2639999985694885, 0.44200000166893005, 0.03999999910593033, 0.9789999723434448, 0.05900000035762787, 0.328000009059906, 0.12700000405311584, 0.921999990940094, -0.027000000700354576, 0.00800000037997961, 0.07199999690055847, -0.02800000086426735, 0.8450000286102295, 0.12200000137090683, 0.11900000274181366, 0.8220000267028809], "pred_arousal": [-0.13899999856948853, 0.9369999766349792, 0.5379999876022339, -0.32499998807907104, 0.828000009059906, 0.050999999046325684, 0.06499999761581421, 0.1689999

{"emotion_correct": 2, "valence_loss": 0.7033761739730835, "arousal_loss": 0.5722302794456482, "emotion_loss": NaN, "time_take": 5.036875486373901, "pred_valence": [-0.4440000057220459, -0.671999990940094, -0.4560000002384186, -0.25600001215934753, -0.36800000071525574, -0.22599999606609344, -0.08900000154972076, -0.49900001287460327, -0.3869999945163727, -0.5070000290870667, -0.29100000858306885, -0.5960000157356262, -0.41999998688697815, -0.6290000081062317, -0.33500000834465027, -0.4320000112056732, -0.3050000071525574, -0.15399999916553497, -0.3479999899864197, -0.30000001192092896, -0.9269999861717224, -0.1679999977350235, -0.6899999976158142, -0.3440000116825104, -0.13699999451637268, -0.19499999284744263, -0.5090000033378601, -0.33399999141693115, -0.37299999594688416, -0.546999990940094, -0.296999990940094, -0.27000001072883606], "pred_arousal": [-0.8820000290870667, -0.8809999823570251, -0.5950000286102295, -0.1850000023841858, -0.42800000309944153, -0.46799999475479126, -0.50

{"emotion_correct": 1, "valence_loss": 1.390802025794983, "arousal_loss": 1.0416399240493774, "emotion_loss": NaN, "time_take": 6.272145986557007, "pred_valence": [-0.5210000276565552, -0.4230000078678131, -0.7770000100135803, -0.5, -0.3050000071525574, -0.4740000069141388, -0.3089999854564667, -0.5450000166893005, -0.6000000238418579, -0.7540000081062317, -0.5320000052452087, -0.3630000054836273, -0.5009999871253967, -0.33500000834465027, -0.39800000190734863, -0.5220000147819519, -0.3440000116825104, -0.4909999966621399, -0.3659999966621399, -0.3659999966621399, -0.2759999930858612, -0.4350000023841858, -0.4300000071525574, -0.5210000276565552, -0.4729999899864197, -0.3199999928474426, -0.3479999899864197, -0.4020000100135803, -0.5170000195503235, -0.45899999141693115, -0.9390000104904175, -0.4790000021457672], "pred_arousal": [-0.8450000286102295, -0.164000004529953, -0.7020000219345093, -0.5320000052452087, -0.4779999852180481, -0.5569999814033508, -0.5370000004768372, -0.532000005

In [ ]:
log